<a href="https://colab.research.google.com/github/jasxn808/Basic-Python-Analysis-NFL-Stats/blob/main/Week4_PHI_WAS_runPercentage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nfl_data_py

In [ ]:
import pandas as pd
import nfl_data_py as nfl

In [ ]:
df = nfl.import_pbp_data([2023])

2023 done.
Downcasting floats.


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#pbp_2021.query('play_type == "pass" & air_yards.notnull()')
df = df.query('game_id == "2023_04_WAS_PHI" & posteam.notnull() & yards_gained.notnull() & (play_type == "run" | play_type == "pass")')

df = df.loc[:, ['game_id', 'play_id', 'posteam', 'defteam', 'yards_gained', 'play_type', 'offense_personnel']]

In [ ]:
##basic aggregation:

#median yards_gained
df['yards_gained'].median()

3.0

In [ ]:
#df.median()
#df.describe()

,play_id,yards_gained
count,142.000000,142.000000
mean,2425.119629,5.507042
std,1443.330078,8.760604
min,55.000000,-8.000000
25%,1186.250000,0.000000
50%,2420.000000,3.000000
75%,3669.500000,8.000000
max,4879.000000,59.000000


In [ ]:
#df['yards_gained'].value_counts()
#df['play_type'].value_counts()
df['offense_personnel'].value_counts()
#df['offense_personnel'].value_counts(normalize=True)

1 RB, 1 TE, 3 WR    120
1 RB, 2 TE, 2 WR     18
2 RB, 1 TE, 2 WR      2
1 RB, 3 TE, 1 WR      2
Name: offense_personnel, dtype: int64

In [ ]:
##group by offense_personnel
formation_grp = df.groupby(['offense_personnel'])


In [ ]:
df.query('offense_personnel == "1 RB, 1 TE, 3 WR"')['yards_gained'].value_counts()

In [ ]:
#formation_grp['yards_gained'].value_counts()
formation_grp['yards_gained'].value_counts(normalize=True).loc['1 RB, 2 TE, 2 WR']

yards_gained
0.0     0.277778
2.0     0.222222
9.0     0.111111
1.0     0.055556
3.0     0.055556
5.0     0.055556
6.0     0.055556
8.0     0.055556
13.0    0.055556
17.0    0.055556
Name: yards_gained, dtype: float64

In [ ]:
## aggregates: mean, median, etc
#formation_grp['yards_gained'].median()
formation_grp['yards_gained'].agg(['median', 'mean'])

#formation_grp['yards_gained'].agg(['median', 'mean']).loc['1 RB, 1 TE, 3 WR']

,median,mean
offense_personnel,,
"1 RB, 1 TE, 3 WR",3.0,5.775000
"1 RB, 2 TE, 2 WR",2.0,4.388889
"1 RB, 3 TE, 1 WR",1.5,1.500000
"2 RB, 1 TE, 2 WR",3.5,3.500000


In [ ]:
#summing boolean count
df.loc[df['play_type'] == 'run']['offense_personnel'].str.contains('1 TE').sum()  ##46 run plays where 1 TE included


46

In [ ]:
formation_grp['play_type'].apply(lambda x: x.str.contains('run').sum())   #count of number of run plays in each personnel

offense_personnel
1 RB, 1 TE, 3 WR    45
1 RB, 2 TE, 2 WR    10
1 RB, 3 TE, 1 WR     0
2 RB, 1 TE, 2 WR     1
Name: play_type, dtype: int64

In [ ]:
formation_grp['play_type'].apply(lambda x: x.str.contains('pass').sum()) #count of number of pass plays in each personnel

offense_personnel
1 RB, 1 TE, 3 WR    75
1 RB, 2 TE, 2 WR     8
1 RB, 3 TE, 1 WR     2
2 RB, 1 TE, 2 WR     1
Name: play_type, dtype: int64

In [ ]:
## % of run plays from each personnel out of total plays ran from personnel
plays_formation = df['offense_personnel'].value_counts()
runs_formation = formation_grp['play_type'].apply(lambda x: x.str.contains('run').sum())

python_df = pd.concat([plays_formation, runs_formation], axis='columns', sort=False)
python_df.rename(columns={'offense_personnel':'total_plays', 'play_type':'run_plays'}, inplace=True)

python_df['pct_run'] = (python_df['run_plays'] / python_df['total_plays']) * 100
python_df

,total_plays,run_plays,pct_run
"1 RB, 1 TE, 3 WR",120,45,37.500000
"1 RB, 2 TE, 2 WR",18,10,55.555556
"2 RB, 1 TE, 2 WR",2,1,50.000000
"1 RB, 3 TE, 1 WR",2,0,0.000000


In [ ]:
python_df.sort_values('pct_run', ascending=False, inplace=True)
python_df

,total_plays,run_plays,pct_run
"1 RB, 2 TE, 2 WR",18,10,55.555556
"2 RB, 1 TE, 2 WR",2,1,50.000000
"1 RB, 1 TE, 3 WR",120,45,37.500000
"1 RB, 3 TE, 1 WR",2,0,0.000000
